In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# load training data
train = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/IMDB_Dataset.csv', encoding='iso-8859-1')
train.shape

(50000, 2)

In [4]:
train =train.sample(frac =.002)
train.shape

(100, 2)

In [5]:
import re
# clean noise from text
def clean_text(text):
    # filter to allow only alphabets
    text = re.sub(r'[^a-zA-Z\']', ' ', text)

    # remove Unicode characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)

    # convert to lowercase to maintain consistency
    text = text.lower()

    return text


In [6]:
train['clean_text'] = train.review.apply(clean_text)

In [7]:
from sklearn.model_selection import train_test_split

# split into training and validation sets
X_tr, X_val, y_tr, y_val = train_test_split(train.clean_text, train.sentiment, test_size=0.25, random_state=42)


In [8]:
print('X_tr shape:',X_tr.shape)

X_tr shape: (75,)


In [9]:
!pip install bert-serving-client
!pip install -U bert-serving-server[http]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.2/357.2 kB 14.6 MB/s eta 0:00:00
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7408 sha256=4c8a3f9184343eedcb65263de3c39e10f51554c2c43e44588040adaa0fa43634
  Stored in directory: /root/.cache/pip/wheels/2b/b5/24/fbb56595c286984f7315ee31821d6121e1b9828436021a88b3
Successfully built GPUtil


In [10]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip uncased_L-12_H-768_A-12.zip

--2023-04-13 07:22:40--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.6.128, 108.177.112.128, 74.125.124.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.6.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M   209MB/s    in 1.9s    

2023-04-13 07:22:42 (209 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [11]:
!nohup bert-serving-start -model_dir=./uncased_L-12_H-768_A-12 -num_worker=4 

nohup: ignoring input and appending output to 'nohup.out'


In [12]:
!ls  # you should see uncased_something_.zip

drive	   sample_data		    uncased_L-12_H-768_A-12.zip
nohup.out  uncased_L-12_H-768_A-12


In [ ]:
from bert_serving.client import BertClient
bc = BertClient()

In [ ]:
# get the embedding for train and val sets
text_list1=X_tr.tolist()
X_tr_bert = bc.encode(text_list1)
text_list2=X_val.tolist()
X_val_bert = bc.encode(text_list2)

In [ ]:
from sklearn.linear_model import LogisticRegression
# LR model
model_bert = LogisticRegression(max_iter=7600)
# train
model_bertLR = model_bert.fit(X_tr_bert, y_tr)
# predict
pred_bert = model_bertLR.predict(X_val_bert)

#Check Classification Accuracy
from sklearn.metrics import accuracy_score

print(accuracy_score(y_val, pred_bert))

In [ ]:
import joblib
joblib.dump(model_bertLR,"/content/drive/MyDrive/Colab_Notebooks/SECONDYEAR_TRIMESTER4_Module03_deep-learning-applications/DAY1_APPLICATION_OF_DEEP_LEARNING/imdbBertLR.model")
model_bertLR = joblib.load("/content/drive/MyDrive/Colab_Notebooks/SECONDYEAR_TRIMESTER4_Module03_deep-learning-applications/DAY1_APPLICATION_OF_DEEP_LEARNING/imdbBertLR.model")


In [ ]:
#SVM Model
from sklearn import svm
model_bert = svm.SVC(decision_function_shape='ovo')

In [ ]:
# train
model_bertSVM = model_bert.fit(X_tr_bert, y_tr)
# predict
pred_bert = model_bertSVM.predict(X_val_bert)

#Check Classification Accuracy
from sklearn.metrics import accuracy_score

print(accuracy_score(y_val, pred_bert))


In [ ]:
joblib.dump(model_bertSVM,"/content/drive/MyDrive/Colab_Notebooks/SECONDYEAR_TRIMESTER4_Module03_deep-learning-applications/DAY1_APPLICATION_OF_DEEP_LEARNING/imdbBertSVM.model")
model_bertSVM = joblib.load("/content/drive/MyDrive/Colab_Notebooks/SECONDYEAR_TRIMESTER4_Module03_deep-learning-applications/DAY1_APPLICATION_OF_DEEP_LEARNING/imdbBertSVM.model")


In [ ]:
!jupyter nbconvert --to html Bert_IMDB_Classification.ipynb